In [2]:
import importlib
from datetime import datetime, timedelta

from poly_api import poly_client


In [2]:
ss_markets = poly_client.get_simplified_markets()
ss_markets["data"][0]

{'condition_id': '0x9deb0baac40648821f96f01339229a422e2f5c877de55dc4dbf981f95a1e709c',
 'rewards': {'rates': None, 'min_size': 0, 'max_spread': 0},
 'tokens': [{'token_id': '14270523446080509320829200481895961480205553513304203367521919818541658424782',
   'outcome': 'Chiefs',
   'price': 1,
   'winner': False},
  {'token_id': '93110170397161149119544349457822484949376809039410140245101963942162463626903',
   'outcome': 'Raiders',
   'price': 0,
   'winner': False}],
 'active': True,
 'closed': True,
 'archived': False,
 'accepting_orders': False}

In [3]:
import time


def find_crypto_markets(limit=10, max_queries=3, cursor="MA==", condition=lambda m: m["tags"] and "Crypto" in m["tags"]):
    found_any = False
    crypto_markets = []

    queries_made = 0
    while (queries_made < max_queries) and (len(crypto_markets) < limit):
        print(f"Fetching markets with cursor {cursor}")

        markets = poly_client.get_markets(next_cursor=cursor)
        cursor = markets["next_cursor"]

        print(f"Loaded {len(markets["data"])} markets")

        data = [m for m in markets["data"] if condition(m)]
        if data:
            if not found_any:
                print(f"First cursor with data found: {cursor}")
                found_any = True

        print(f"Found {len(data)} crypto markets")

        crypto_markets += data
        queries_made += 1

        time.sleep(0.5)

    markets = poly_client.get_markets()

    if not crypto_markets:
        print("No crypto markets found")
    else:
        print(f"Crypto markets found ({len(crypto_markets)})")
    
    print(f"Next cursor: {cursor}")

    return crypto_markets


def is_btc_related_market(m):
    return "market_slug" in m.keys() and ("btc" in m["market_slug"])

# def is_

markets = find_crypto_markets(limit=10, cursor="OTAwMA==", condition=is_btc_related_market)

Fetching markets with cursor OTAwMA==
Loaded 500 markets
First cursor with data found: OTUwMA==
Found 3 crypto markets
Fetching markets with cursor OTUwMA==
Loaded 500 markets
Found 2 crypto markets
Fetching markets with cursor MTAwMDA=
Loaded 500 markets
Found 1 crypto markets
Crypto markets found (6)
Next cursor: MTA1MDA=


In [4]:
[("https://polymarket.com/event/" + m["market_slug"], m["condition_id"]) for m in markets]


[('https://polymarket.com/event/will-btc-reach-70k-in-july',
  '0x7c1a2610911ea83b5e9a44d89201635c69919324d45e8deb92ef422b1dddc12b'),
 ('https://polymarket.com/event/will-btc-reach-80k-in-july',
  '0x10bfd2053bfe798aa0fec72fc4aaf465b17b60afb453919ad2f366770042a8d7'),
 ('https://polymarket.com/event/will-btc-reach-75k-in-july',
  '0x409bbbd78d6bf45cd67048fd9ef80f473152f82aef9f3dd275a616af7474cab7'),
 ('https://polymarket.com/event/will-btc-hit-50k-or-70k-first',
  '0xf2830173b5e239e7db0509d4489105107225101cce05b41c1818b4b78e00d6e8'),
 ('https://polymarket.com/event/will-btc-or-eth-reach-all-time-high-first-2024',
  ''),
 ('https://polymarket.com/event/will-a-presidential-candidate-say-mog-at-btc-conference',
  '0x69e439c06b8ce7575d2dcb209a79a362981ec2c949cee41123e5c248807b9ed1')]

In [18]:
market = poly_client.get_market("0x9d84821a6c8b45fcd9dad9f50f1b0fc6cb76de7a68d7686bfefba697c32a6375")
market["description"], market["tokens"]

('This market will immediately resolve to "Yes" if any Binance 1 minute candle for Bitcoin (BTCUSDT) between April 1, 2025, 00:00 and April 30, 2025, 23:59 in the ET timezone has a final "High" price of $95,000 or higher. Otherwise, this market will resolve to "No."\n\nThe resolution source for this market is Binance, specifically the BTCUSDT "High" prices available at https://www.binance.com/en/trade/BTC_USDT, with the chart settings on "1m" for one-minute candles selected on the top bar.\n\nPlease note that the outcome of this market depends solely on the price data from the Binance BTCUSDT trading pair. Prices from other exchanges, different trading pairs, or spot markets will not be considered for the resolution of this market.',
 [{'token_id': '113733303390103764159442829422332472444843638571953553342010572782499227977688',
   'outcome': 'Yes',
   'price': 0.14,
   'winner': False},
  {'token_id': '63580689056305536048863052547924937121791141239998704107552406492946329546681',
   

In [19]:
from py_clob_client.client import BookParams

books = poly_client.get_order_books([BookParams("113733303390103764159442829422332472444843638571953553342010572782499227977688"), BookParams("63580689056305536048863052547924937121791141239998704107552406492946329546681")])
books

[OrderBookSummary(market='0x9d84821a6c8b45fcd9dad9f50f1b0fc6cb76de7a68d7686bfefba697c32a6375', asset_id='113733303390103764159442829422332472444843638571953553342010572782499227977688', timestamp='1744954994671', bids=[OrderSummary(price='0.01', size='11638.66'), OrderSummary(price='0.02', size='15158.56'), OrderSummary(price='0.03', size='15000'), OrderSummary(price='0.04', size='17190.47'), OrderSummary(price='0.05', size='5313'), OrderSummary(price='0.06', size='16011'), OrderSummary(price='0.07', size='3522.04'), OrderSummary(price='0.08', size='4971'), OrderSummary(price='0.09', size='4919'), OrderSummary(price='0.1', size='3911'), OrderSummary(price='0.11', size='2410'), OrderSummary(price='0.12', size='3519'), OrderSummary(price='0.13', size='1620.69')], asks=[OrderSummary(price='0.8', size='500'), OrderSummary(price='0.79', size='760'), OrderSummary(price='0.7', size='333'), OrderSummary(price='0.69', size='100'), OrderSummary(price='0.49', size='12'), OrderSummary(price='0.41'

In [25]:
sorted(books[0].bids, key=lambda x: x.price), sorted(books[0].asks, key=lambda x: x.price)

([OrderSummary(price='0.01', size='11638.66'),
  OrderSummary(price='0.02', size='15158.56'),
  OrderSummary(price='0.03', size='15000'),
  OrderSummary(price='0.04', size='17190.47'),
  OrderSummary(price='0.05', size='5313'),
  OrderSummary(price='0.06', size='16011'),
  OrderSummary(price='0.07', size='3522.04'),
  OrderSummary(price='0.08', size='4971'),
  OrderSummary(price='0.09', size='4919'),
  OrderSummary(price='0.1', size='3911'),
  OrderSummary(price='0.11', size='2410'),
  OrderSummary(price='0.12', size='3519'),
  OrderSummary(price='0.13', size='1620.69')],
 [OrderSummary(price='0.15', size='50'),
  OrderSummary(price='0.16', size='4177.48'),
  OrderSummary(price='0.17', size='23428.87'),
  OrderSummary(price='0.18', size='4400'),
  OrderSummary(price='0.19', size='2400'),
  OrderSummary(price='0.2', size='3410'),
  OrderSummary(price='0.21', size='2400'),
  OrderSummary(price='0.22', size='4900'),
  OrderSummary(price='0.23', size='2400'),
  OrderSummary(price='0.24', s

In [7]:
import poly_api

importlib.reload(poly_api)

condition_id = "0x906a2b90f30c62e09dce56fababafafbc4338cb8dac94ac6c3038327d0447180"

ts_options = poly_api.TSOptions(
    60,
    (datetime.fromisoformat("2025-01-15"), datetime.fromisoformat("2025-01-29")),
)
df = poly_api.get_poly_timeseries(
    condition_id,
    ts_options,
)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 336 entries, 2025-01-14 23:00:00 to 2025-01-28 22:00:00
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Yes     336 non-null    float64
 1   No      336 non-null    float64
dtypes: float64(2)
memory usage: 7.9 KB


In [8]:
df.head()

,Yes,No
date,,
2025-01-14 23:00:00,0.0095,0.9905
2025-01-15 00:00:00,0.0095,0.9905
2025-01-15 01:00:00,0.0095,0.9905
2025-01-15 02:00:00,0.0095,0.9905
2025-01-15 03:00:00,0.0100,0.9900


In [ ]:
market = poly_client.get_market("0x9d84821a6c8b45fcd9dad9f50f1b0fc6cb76de7a68d7686bfefba697c32a6375")
market

{'enable_order_book': True,
 'active': True,
 'closed': False,
 'archived': False,
 'accepting_orders': True,
 'accepting_order_timestamp': '2025-04-01T00:02:27Z',
 'minimum_order_size': 5,
 'minimum_tick_size': 0.01,
 'condition_id': '0x9d84821a6c8b45fcd9dad9f50f1b0fc6cb76de7a68d7686bfefba697c32a6375',
 'question_id': '0x6d1c56a18894833ddb876dde83ac4e3871522e06397e6f90f8e3019455a46e5f',
 'question': 'Will Bitcoin reach $95k in April?',
 'description': 'This market will immediately resolve to "Yes" if any Binance 1 minute candle for Bitcoin (BTCUSDT) between April 1, 2025, 00:00 and April 30, 2025, 23:59 in the ET timezone has a final "High" price of $95,000 or higher. Otherwise, this market will resolve to "No."\n\nThe resolution source for this market is Binance, specifically the BTCUSDT "High" prices available at https://www.binance.com/en/trade/BTC_USDT, with the chart settings on "1m" for one-minute candles selected on the top bar.\n\nPlease note that the outcome of this market de

In [35]:
import poly_dataset_loader
importlib.reload(poly_dataset_loader)

tokens, timestamp, asks, bids = poly_dataset_loader.load_order_book(["0x9d84821a6c8b45fcd9dad9f50f1b0fc6cb76de7a68d7686bfefba697c32a6375"])

In [36]:
tokens

[TokenTable(token='113733303390103764159442829422332472444843638571953553342010572782499227977688', market_id='0x9d84821a6c8b45fcd9dad9f50f1b0fc6cb76de7a68d7686bfefba697c32a6375', token_type='Yes'),
 TokenTable(token='63580689056305536048863052547924937121791141239998704107552406492946329546681', market_id='0x9d84821a6c8b45fcd9dad9f50f1b0fc6cb76de7a68d7686bfefba697c32a6375', token_type='No')]

In [38]:
asks, bids

([OrderItem(token='113733303390103764159442829422332472444843638571953553342010572782499227977688', price='0.15', size='4617.48'),
  OrderItem(token='113733303390103764159442829422332472444843638571953553342010572782499227977688', price='0.16', size='3879'),
  OrderItem(token='113733303390103764159442829422332472444843638571953553342010572782499227977688', price='0.17', size='6577.87'),
  OrderItem(token='63580689056305536048863052547924937121791141239998704107552406492946329546681', price='0.86', size='2800'),
  OrderItem(token='63580689056305536048863052547924937121791141239998704107552406492946329546681', price='0.87', size='4120.69'),
  OrderItem(token='63580689056305536048863052547924937121791141239998704107552406492946329546681', price='0.88', size='3778')],
 [OrderItem(token='113733303390103764159442829422332472444843638571953553342010572782499227977688', price='0.12', size='3778'),
  OrderItem(token='113733303390103764159442829422332472444843638571953553342010572782499227977688

In [ ]:
from py_clob_client.client import BookParams

tokens = ["63580689056305536048863052547924937121791141239998704107552406492946329546681"]

order_book = poly_client.get_order_books([BookParams(t) for t in tokens])[0]

[OrderSummary(price='0.85', size='4717.48'),
 OrderSummary(price='0.84', size='3879'),
 OrderSummary(price='0.83', size='15987.87'),
 OrderSummary(price='0.82', size='4410'),
 OrderSummary(price='0.81', size='2400'),
 OrderSummary(price='0.8', size='3410'),
 OrderSummary(price='0.79', size='2400'),
 OrderSummary(price='0.78', size='4900'),
 OrderSummary(price='0.77', size='14400'),
 OrderSummary(price='0.76', size='2400'),
 OrderSummary(price='0.75', size='2493.33'),
 OrderSummary(price='0.74', size='2500'),
 OrderSummary(price='0.73', size='3400'),
 OrderSummary(price='0.72', size='3900'),
 OrderSummary(price='0.71', size='2400'),
 OrderSummary(price='0.7', size='4910'),
 OrderSummary(price='0.69', size='10'),
 OrderSummary(price='0.67', size='4000'),
 OrderSummary(price='0.66', size='10'),
 OrderSummary(price='0.65', size='10'),
 OrderSummary(price='0.63', size='5000'),
 OrderSummary(price='0.6', size='382.41'),
 OrderSummary(price='0.59', size='13'),
 OrderSummary(price='0.51', size

In [26]:

sorted(order_book.bids, key=lambda x: x.price)


[OrderSummary(price='0.01', size='12000'),
 OrderSummary(price='0.2', size='500'),
 OrderSummary(price='0.21', size='760'),
 OrderSummary(price='0.3', size='333'),
 OrderSummary(price='0.31', size='100'),
 OrderSummary(price='0.51', size='12'),
 OrderSummary(price='0.59', size='13'),
 OrderSummary(price='0.6', size='382.41'),
 OrderSummary(price='0.63', size='5000'),
 OrderSummary(price='0.65', size='10'),
 OrderSummary(price='0.66', size='10'),
 OrderSummary(price='0.67', size='4000'),
 OrderSummary(price='0.69', size='10'),
 OrderSummary(price='0.7', size='4910'),
 OrderSummary(price='0.71', size='2400'),
 OrderSummary(price='0.72', size='3900'),
 OrderSummary(price='0.73', size='3400'),
 OrderSummary(price='0.74', size='2500'),
 OrderSummary(price='0.75', size='2493.33'),
 OrderSummary(price='0.76', size='2400'),
 OrderSummary(price='0.77', size='14400'),
 OrderSummary(price='0.78', size='4900'),
 OrderSummary(price='0.79', size='2400'),
 OrderSummary(price='0.8', size='3410'),
 Ord